In [1]:
# #install packages
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.9 MB/s eta 0:00:00


In [2]:
!pip install nltk emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=3681c70a64ca964c3d7f0018973c55aa73dfff55a69437346f4e0e28fd810401
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [28]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from TweetNormalizer import normalizeTweet
from tqdm import tqdm
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.special import expit
from sklearn import metrics
from sklearn.metrics import classification_report

In [9]:
# dataset = pd.read_csv('drive/MyDrive/train_modified.csv')
# dataset.head(2)

train = pd.read_csv('drive/MyDrive/train_df.csv')
test = pd.read_csv('drive/MyDrive/val_df.csv')
train.head()

,Unnamed: 0.1,Unnamed: 0,ID,tweet,labels,label_list,label_len,tweet_len,clean_tweets,tokens,neg,neu,pos,compound
0,1477,1477,1352994869556350977t,#TRUSTCCP? Do you trust the Chinese Communist ...,country,['country'],1,32,trustccp do you trust the chinese communist pa...,"['trustccp', 'trust', 'chinese', 'communist', ...",0.035,0.679,0.286,0.8738
1,1790,1790,1364574998397132805t,@hannah62687 @mgtalbott @DrTomFrieden For many...,ineffective unnecessary,"['ineffective', 'unnecessary']",2,50,hannah62687 mgtalbott drtomfrieden for many ge...,"['hannah62687', 'mgtalbott', 'drtomfrieden', '...",0.139,0.756,0.105,-0.2295
2,2615,2615,1334085299945807873t,Imagine if the covid vaccine has a side effect...,side-effect,['side-effect'],1,17,imagine if the covid vaccine has a side effect...,"['imagine', 'covid', 'vaccine', 'side', 'effec...",0.000,1.000,0.000,0.0000
3,6969,6969,1321645107183980544t,"@GreyHaired07 and yet, every other year we are...",mandatory,['mandatory'],1,31,greyhaired07 and yet every other year we are r...,"['greyhaired07', 'yet', 'every', 'year', 'remi...",0.151,0.849,0.000,-0.5859
4,7172,7172,1432922183337066498t,#VAERS 17y♂️#Pfizer #Covidvaccine #Suicide att...,side-effect,['side-effect'],1,45,vaers 17ypfizer covidvaccine suicide attempt p...,"['vaers', '17ypfizer', 'covidvaccine', 'suicid...",0.092,0.857,0.051,-0.4588


In [11]:
# dataset['tweet_normalized'] = dataset['tweet'].apply(lambda x: normalizeTweet(x))
# dataset.head(2)

train['tweet_normalized'] = train['tweet'].apply(lambda x: normalizeTweet(x))
test['tweet_normalized'] = test['tweet'].apply(lambda x: normalizeTweet(x))
train.head(2)

,Unnamed: 0.1,Unnamed: 0,ID,tweet,labels,label_list,label_len,tweet_len,clean_tweets,tokens,neg,neu,pos,compound,tweet_normalized
0,1477,1477,1352994869556350977t,#TRUSTCCP? Do you trust the Chinese Communist ...,country,['country'],1,32,trustccp do you trust the chinese communist pa...,"['trustccp', 'trust', 'chinese', 'communist', ...",0.035,0.679,0.286,0.8738,#TRUSTCCP ? Do you trust the Chinese Communist...
1,1790,1790,1364574998397132805t,@hannah62687 @mgtalbott @DrTomFrieden For many...,ineffective unnecessary,"['ineffective', 'unnecessary']",2,50,hannah62687 mgtalbott drtomfrieden for many ge...,"['hannah62687', 'mgtalbott', 'drtomfrieden', '...",0.139,0.756,0.105,-0.2295,"@USER @USER @USER For many , getting severe CO..."


In [13]:
# y_dataset = dataset['label_list'].apply(ast.literal_eval)

y_train = train['label_list'].apply(ast.literal_eval)
y_test = test['label_list'].apply(ast.literal_eval)

In [14]:
# Binarize the labels for multi-label classification
mlb = MultiLabelBinarizer()
# y_dataset_binarized = mlb.fit_transform(y_dataset)

y_train_binarized = mlb.fit_transform(y_train)
y_test_binarized = mlb.fit_transform(y_test)

In [15]:
# dataset['one_hot_labels'] = list(y_dataset_binarized)
# dataset.head(2)

train['one_hot_labels'] = list(y_train_binarized)
test['one_hot_labels'] = list(y_test_binarized)
train.head(2)

,Unnamed: 0.1,Unnamed: 0,ID,tweet,labels,label_list,label_len,tweet_len,clean_tweets,tokens,neg,neu,pos,compound,tweet_normalized,one_hot_labels
0,1477,1477,1352994869556350977t,#TRUSTCCP? Do you trust the Chinese Communist ...,country,['country'],1,32,trustccp do you trust the chinese communist pa...,"['trustccp', 'trust', 'chinese', 'communist', ...",0.035,0.679,0.286,0.8738,#TRUSTCCP ? Do you trust the Chinese Communist...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1790,1790,1364574998397132805t,@hannah62687 @mgtalbott @DrTomFrieden For many...,ineffective unnecessary,"['ineffective', 'unnecessary']",2,50,hannah62687 mgtalbott drtomfrieden for many ge...,"['hannah62687', 'mgtalbott', 'drtomfrieden', '...",0.139,0.756,0.105,-0.2295,"@USER @USER @USER For many , getting severe CO...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [19]:
# from sklearn.model_selection import train_test_split

# # Split the DataFrame into train and validation sets
# train_df, test_df = train_test_split(dataset, test_size=0.2, random_state=42)

train_df, test_df = train, test
print(train_df.shape)
print(test_df.shape)

(7936, 16)
(1985, 16)


In [17]:
MODEL_PATH = 'vinai/bertweet-large'
MAX_LENGTH = 128

In [20]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [21]:
train_encodings = tokenizer(train_df["tweet_normalized"].values.tolist(), max_length=MAX_LENGTH, padding="max_length",truncation=True,return_tensors='pt')
test_encodings = tokenizer(test_df["tweet_normalized"].values.tolist(), max_length=MAX_LENGTH, padding="max_length",truncation=True,return_tensors='pt')

In [22]:
train_labels = train_df["one_hot_labels"].values.tolist()
test_labels = test_df["one_hot_labels"].values.tolist()

In [23]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # labels need to be converted to float for multilabel loss calculation
        item['labels'] = torch.tensor(self.labels[idx]).float()
        return item

    def __len__(self):
        return len(self.labels)

In [24]:
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [25]:
train_dataset[0]

<ipython-input-23-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([    0, 10431,  6997, 11120,  3376,   510, 17487,  1832,    47,  2416,
             5,  1111, 12416,  1643,     7,   146, 21688, 47412,   128,  9937,
           128, 17487,  2647,  2156,   205,  2156,  1303,    89,    32,  2213,
             9,    49, 12987, 34426,  1952,   447,   543,    13,    47,   235,
           122,    11,  1776, 17914,  1666, 44454, 42703,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [26]:
# sanity check
tokenizer.decode(train_dataset[0]["input_ids"])

<ipython-input-23-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


"<s>#TRUSTCCP? Do you trust the Chinese Communist Party to make YOUR mRNA'vaccine '? Well, good, cause there are hundreds of their virologists working hard for you right now in Big Pharma... HTTPURL</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>"

In [27]:
# id2label = { 0: "unnecessary", 1: "mandatory", 2: "pharma", 3 : "conspiracy", 4:"political", 5:"country", 6:"rushed", 7:"ingredients", 8:"side-effect", 9:"ineffective", 10:"religious", 11:"none"}
# label2id = { "unnecessary": 0, "mandatory": 1, "pharma": 2, "conspiracy": 3,  "political": 4, "country": 5, "rushed": 6, "ingredients": 7, "side-effect":8, "ineffective": 9, "religious": 10, "none": 11}
label_cols=list(mlb.classes_)
label2id = dict()
id2label = dict()
for idx, col in enumerate(label_cols):
    id2label[idx] = col
    label2id[col] = idx

In [29]:
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.array(expit(preds) > thresh, dtype=int)
    macro_f1 = metrics.f1_score(labels, preds, average='macro', zero_division=0)
    micro_f1 = metrics.f1_score(labels, preds, average='micro', zero_division=0)
    weighted_f1 = metrics.f1_score(labels, preds, average='weighted', zero_division=0)
    samples_f1 = metrics.f1_score(labels, preds, average='samples', zero_division=0)
    acc_thresh = accuracy_thresh(preds, labels, thresh)
    return {"macro_f1":macro_f1, "micro_f1":micro_f1,
            "weighted_f1":weighted_f1, "samples_f1":samples_f1,
            'accuracy_thresh': acc_thresh}

def accuracy_thresh(y_pred, y_true, thresh, sigmoid=True):
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.bool()).float().mean().item()

def get_binary_preds(y_probs, thresh):
    if y_probs > thresh:
        return 1
    else:
        return 0
get_binary_preds_vec = np.vectorize(get_binary_preds)

def generate_performance_metrics(test_dataset,label_classes):
    predictions = trainer.predict(test_dataset)
    all_preds = predictions.predictions
    all_preds = torch.from_numpy(all_preds).float()
    preds = all_preds.sigmoid()

#     thresh = args.thresh

    test_bin_preds = get_binary_preds_vec(preds.numpy(), thresh)

    tdf_1hot = pd.DataFrame(predictions.label_ids, columns=label_classes)
    labels_subset_ids = [label2id[c] for c in label_classes if tdf_1hot[c].sum()>0]
    labels_subset = [c for c in label_classes if tdf_1hot[c].sum()>0]

    all_metrics = classification_report(predictions.label_ids, test_bin_preds, digits=3, labels = labels_subset_ids,
                                       target_names=labels_subset, output_dict=True, zero_division=0)
    metrics_df = pd.DataFrame(all_metrics).transpose().reset_index()
    return metrics_df

In [30]:
num_labels = len(id2label)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=num_labels, label2id=label2id, id2label=id2label , problem_type="multi_label_classification")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
batch_size = 10
# configure logging so we see training loss
logging_steps = len(train_dataset) // batch_size
args = TrainingArguments(
    output_dir="bertweet",
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    learning_rate=2e-5,
    fp16=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    #weight_decay=0.01,
    logging_steps=logging_steps,
    gradient_accumulation_steps= 32,
    lr_scheduler_type='cosine',
    metric_for_best_model='micro_f1',
    load_best_model_at_end = True
)

In [34]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.001)]
)

In [35]:
thresh=0.2
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-23-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Macro F1,Micro F1,Weighted F1,Samples F1,Accuracy Thresh
0,No log,0.286251,0.046320,0.343834,0.172696,0.350210,0.103233
1,No log,0.215249,0.333406,0.575712,0.525651,0.584855,0.103233
2,No log,0.174711,0.500329,0.670812,0.648448,0.687703,0.103233
3,No log,0.152442,0.583070,0.716544,0.712182,0.736190,0.103233
4,No log,0.147220,0.601582,0.724700,0.726016,0.749050,0.103233
5,No log,0.145161,0.611376,0.732307,0.730968,0.753911,0.103233
6,No log,0.143438,0.616020,0.739019,0.738460,0.761456,0.103233
7,No log,0.142841,0.619146,0.739650,0.737382,0.761538,0.103233
8,No log,0.142736,0.615478,0.736784,0.736774,0.759340,0.103233
9,No log,0.142721,0.615625,0.736650,0.736473,0.758904,0.103233


<ipython-input-23-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-23-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-23-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-23-731660984850>:7: UserWarning: To copy construct from a 

TrainOutput(global_step=240, training_loss=0.16605931917826336, metrics={'train_runtime': 2055.1027, 'train_samples_per_second': 38.616, 'train_steps_per_second': 0.117, 'total_flos': 1.7885299079835648e+16, 'train_loss': 0.16605931917826336, 'epoch': 9.67})

In [36]:
test_result = trainer.evaluate()
test_result

<ipython-input-23-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.14284147322177887,
 'eval_macro_f1': 0.6191460930878229,
 'eval_micro_f1': 0.7396504139834407,
 'eval_weighted_f1': 0.7373822025202117,
 'eval_samples_f1': 0.7615377234017032,
 'eval_accuracy_thresh': 0.10323257744312286,
 'eval_runtime': 14.1302,
 'eval_samples_per_second': 140.479,
 'eval_steps_per_second': 14.083,
 'epoch': 9.67}

In [37]:
# label_cols = list(mlb.classes_)

In [38]:
test_performance_df = generate_performance_metrics(test_dataset,label_cols)
test_performance_df

<ipython-input-23-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


,index,precision,recall,f1-score,support
0,conspiracy,0.428571,0.593407,0.497696,91.0
1,country,0.595238,0.641026,0.617284,39.0
2,ineffective,0.690000,0.807018,0.743935,342.0
3,ingredients,0.744000,0.815789,0.778243,114.0
4,mandatory,0.666667,0.838710,0.742857,155.0
5,none,0.596491,0.591304,0.593886,115.0
6,pharma,0.623494,0.802326,0.701695,258.0
7,political,0.468750,0.545455,0.504202,110.0
8,religious,0.000000,0.000000,0.000000,13.0
9,rushed,0.665835,0.895973,0.763948,298.0


In [39]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = trainer.module if hasattr(trainer, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.json',
 './model_save/merges.txt',
 './model_save/added_tokens.json',
 './model_save/tokenizer.json')

In [40]:
!ls -l --block-size=K ./model_save/

total 1391612K
-rw-r--r-- 1 root root       2K Aug 10 15:34 config.json
-rw-r--r-- 1 root root     446K Aug 10 15:35 merges.txt
-rw-r--r-- 1 root root 1388305K Aug 10 15:35 pytorch_model.bin
-rw-r--r-- 1 root root       1K Aug 10 15:35 special_tokens_map.json
-rw-r--r-- 1 root root       1K Aug 10 15:35 tokenizer_config.json
-rw-r--r-- 1 root root    2060K Aug 10 15:35 tokenizer.json
-rw-r--r-- 1 root root     780K Aug 10 15:35 vocab.json


In [41]:
# # Mount Google Drive to this Notebook instance.
# from google.colab import drive
#     drive.mount('/content/drive')

In [42]:
# Copy the model files to a directory in your Google Drive.
!cp -r ./model_save/ "./drive/MyDrive/Colab Notebooks/bertweet_large_finetuned/"

## Inference

In [43]:
# bertweet-large finetuned model
FINETUNED_MODEL_PATH = './drive/MyDrive/Colab Notebooks/bertweet_large_finetuned'
MAX_LENGTH = 128

In [44]:
# id2label = { 0: "unnecessary", 1: "mandatory", 2: "pharma", 3 : "conspiracy", 4:"political", 5:"country", 6:"rushed", 7:"ingredients", 8:"side-effect", 9:"ineffective", 10:"religious", 11:"none"}
# label2id = { "unnecessary": 0, "mandatory": 1, "pharma": 2, "conspiracy": 3,  "political": 4, "country": 5, "rushed": 6, "ingredients": 7, "side-effect":8, "ineffective": 9, "religious": 10, "none": 11}

In [45]:
num_labels = len(id2label)

In [46]:
finetuned_tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL_PATH)

In [47]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained(FINETUNED_MODEL_PATH, num_labels=num_labels, label2id=label2id, id2label=id2label , problem_type="multi_label_classification")

In [48]:
final_test = pd.read_csv('./drive/MyDrive/test.csv')
final_test.head()

,id,tweet
0,1070378532260470789t,Study Links HPV Vaccine to Historically High I...
1,973746711964372993t,Deaths from tainted measles vaccine affecting ...
2,1043031076787040257t,"Am apreciat un videoclip pe @YouTube, https://..."
3,1066338147527741440t,VIDEO --&gt;&gt; MMR #Vaccine increase risk of...
4,963522018544152576t,Oral Polio Vaccine: Infecting Unvaccinated Kid...


In [49]:
final_test['tweet_normalized'] = final_test['tweet'].apply(lambda x: normalizeTweet(x))
final_test.head()

,id,tweet,tweet_normalized
0,1070378532260470789t,Study Links HPV Vaccine to Historically High I...,Study Links HPV Vaccine to Historically High I...
1,973746711964372993t,Deaths from tainted measles vaccine affecting ...,Deaths from tainted measles vaccine affecting ...
2,1043031076787040257t,"Am apreciat un videoclip pe @YouTube, https://...","Am apreciat un videoclip pe @USER , HTTPURL Tr..."
3,1066338147527741440t,VIDEO --&gt;&gt; MMR #Vaccine increase risk of...,VIDEO --> > MMR #Vaccine increase risk of #AUT...
4,963522018544152576t,Oral Polio Vaccine: Infecting Unvaccinated Kid...,Oral Polio Vaccine : Infecting Unvaccinated Ki...


In [50]:
item_list = [{
    'id': row['id'],
    'tweet_normalized': row['tweet_normalized']#,
    #'real_labels': list(row['labels'])
    } for idx, row in final_test.iterrows()]

In [51]:
thresh=0.5
batch_size = 8

In [52]:
def get_batched_predictions(item_list, batch_size, thresh):
    outputs_list = []
    for batch in tqdm([item_list[x:x + batch_size]
                      for x in range(0, len(item_list), batch_size)]):
        # tokenize
        encoded = finetuned_tokenizer(
            [d['tweet_normalized'] for d in batch],  # list of text inputs to encode
            max_length=512,  # Pad & truncate all sentences.
            padding=True,
            truncation=True,
            return_token_type_ids=False,
            return_tensors='pt'
            )

        # predict
        with torch.no_grad():
            outputs = finetuned_model(
                encoded['input_ids'],
                token_type_ids=None,
                attention_mask=encoded['attention_mask']
            )
        probs = np.array(expit(outputs[0]))

        # append predictions
        for i, b in enumerate(batch):
            outputs_list.append({
                'id': b['id'],
                'probabilities': probs[i],
                'predictions': list(map(lambda x: int(x > thresh), probs[i]))#,
               # 'real_labels': b['real_labels']
                })

    return outputs_list

In [53]:
outputs_list = get_batched_predictions(item_list, batch_size, thresh)

100%|██████████| 61/61 [06:02<00:00,  5.95s/it]


In [54]:
predictions_list = dict()
for p in outputs_list:
    predictions_list[p['id']] = [{
        'indices_pred':[i for i, e in enumerate(p['predictions']) if e == 1]#,
        #'indices_real': [i for i, e in enumerate(p['real_labels']) if e == 1],
        }
    ]

In [55]:
out_list = []
for i in tqdm(final_test['id']):
    df_items = {
    'id' : i,
    'pred_classes' : [id2label[k] for k in predictions_list[i][0]['indices_pred']]#,
   # 'actual_classes'  : [id2label[k] for k in predictions_list[i][0]['indices_real']]
    }
    out_list.append(df_items)

100%|██████████| 486/486 [00:00<00:00, 261236.93it/s]


In [56]:
out_df = pd.DataFrame(out_list)

In [57]:
out_df.head()

,id,pred_classes
0,1070378532260470789t,[side-effect]
1,973746711964372993t,[side-effect]
2,1043031076787040257t,[side-effect]
3,1066338147527741440t,[side-effect]
4,963522018544152576t,[ineffective]


In [58]:
out_df.explode(['pred_classes']).groupby(['pred_classes'])['id'].count()

pred_classes
conspiracy      40
country          6
ineffective     59
ingredients     50
mandatory       50
none            23
pharma          55
political       23
rushed          35
side-effect    175
unnecessary     10
Name: id, dtype: int64

In [ ]:
# Function to remove 'none' if it appears with another label
def remove_none(lst):
    if 'none' in lst and len(lst) > 1:
        lst.remove('none')
    return lst

# Apply the function to each list in 'column_name'
out_df['pred_classes'] = out_df['pred_classes'].apply(remove_none)

In [ ]:
out_df.head()

,id,pred_classes
0,1070378532260470789t,[religious]
1,973746711964372993t,[religious]
2,1043031076787040257t,[religious]
3,1066338147527741440t,[religious]
4,963522018544152576t,[pharma]


In [ ]:
out_df['pred_classes'] = out_df['pred_classes'].apply(' '.join)

In [ ]:
out_df[out_df['pred_classes']==''].shape # predicting nothing

(40, 2)

In [ ]:
out_df.to_csv('./drive/MyDrive/bertweet_large_tuning.csv',index=False)